In [0]:
! git clone https://github.com/5amessi/license_plates.git

fatal: destination path 'license_plates' already exists and is not an empty directory.


In [0]:
! git clone https://github.com/SeyedHamidreza/car_plate_dataset.git


fatal: destination path 'car_plate_dataset' already exists and is not an empty directory.


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/car_plate_dataset/IRCP_dataset_1024X768.zip", 'r')
zip_ref.extractall("")
zip_ref.close()

In [0]:
# !pip install kaggle


In [0]:
# !mkdir .kaggle


mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
# import json
# token = {"username":"mohamedkhamis","key":"6c145735268dd0cdb289fa1fc5078506"}
# with open('/content/.kaggle/kaggle.json', 'w') as file:
#     json.dump(token, file)

In [0]:
# !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
# !kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:

# !kaggle datasets download -d thamizhsterio/indian-license-plates


 99% 237M/239M [00:01<00:00, 219MB/s]
100% 239M/239M [00:01<00:00, 230MB/s]


In [0]:
# import zipfile
# path = "/content/{/content}/datasets/thamizhsterio/indian-license-plates/indian-license-plates.zip"
# zip_ref = zipfile.ZipFile(path, 'r')
# zip_ref.extractall("")
# zip_ref.close()

In [0]:
# import zipfile
# path = "/content/Indian Number Plate.zip"
# zip_ref = zipfile.ZipFile(path, 'r')
# zip_ref.extractall("")
# zip_ref.close()

In [0]:
import scipy
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import *
from keras.layers import *


class DataLoader():
    def __init__(self, dataset_name, img_res=(480, 240),out_res=(480, 240)):
        self.dataset_name = dataset_name
        self.img_res = img_res
        self.out_res = out_res

    def load_data(self,png = False):
        if png == True:
          path = glob('/content/%s/*.png' % (self.dataset_name))
        else:
          path = glob('/content/%s/*' % (self.dataset_name))
        
        imgs_hr = []
        imgs_lr = []

        
        for idx , i in enumerate(path):
          if idx >= 1000:
            break
          img = cv2.imread(i)
          w, h = self.img_res
          low_w, low_h = int(w / 4), int(h / 4)

          img_hr = cv2.resize(img, self.out_res)
          
          img_lr = cv2.resize(img, (low_w, low_h))
          img_lr = cv2.resize(img_lr, self.img_res)

          flr=np.fliplr(img_lr)
          fhr=np.fliplr(img_hr)

          imgs_hr.append(img_hr)
          imgs_hr.append(fhr)
          imgs_lr.append(img_lr)
          imgs_lr.append(flr)    

        imgs_hr = np.array(imgs_hr) / 127.5 - 1.
        imgs_lr = np.array(imgs_lr) / 127.5 - 1.

        return imgs_hr, imgs_lr

dl = DataLoader("license_plates")
hr , lr = dl.load_data()
print(np.shape(hr))
print(np.shape(lr))

dl = DataLoader("IRCP_dataset_1024X768")
hr2 , lr2 = dl.load_data()

print(np.shape(hr2))
print(np.shape(lr2))
hr = np.concatenate((hr, hr2))
lr = np.concatenate((lr, lr2))
# dl = DataLoader("Indian Number Plate")
# hr2 , lr2 = dl.load_data(png = True)
# print(np.shape(hr2))
# print(np.shape(lr2))

# hr = np.concatenate((hr, hr2))
# lr = np.concatenate((lr, lr2))

print(np.shape(hr))
print(np.shape(lr))
t1 = (hr[2] + 1) * 127.5
t1 = np.array(t1, dtype=np.uint8)
cv2.imwrite('hr.jpg',t1)
t1 = (lr[2] + 1) * 127.5
t1 = np.array(t1, dtype=np.uint8)
cv2.imwrite('lr.jpg',t1)


Using TensorFlow backend.


(220, 240, 480, 3)
(220, 240, 480, 3)
(440, 240, 480, 3)
(440, 240, 480, 3)
(660, 240, 480, 3)
(660, 240, 480, 3)


True

In [0]:
def load_image_test(img):
  imgs_lr = []
  w, h = 480 , 240

  img_lr = cv2.resize(img, (w,h))
  
  imgs_lr.append(img_lr)
  imgs_lr = np.array(imgs_lr) / 127.5 - 1.

  return imgs_lr

In [0]:
def pred(count = 0):
  test = cv2.imread("/content/test.jpg")
  test = load_image_test(test)
  result = model.predict([test])
  result = (result + 1) * 127.5
  result = np.array(result, dtype=np.uint8)
  cv2.imwrite("testout%d.jpg"%(count),result[0])
  
  result = model.predict([[lr[0]]])
  result = (result + 1) * 127.5
  result = np.array(result, dtype=np.uint8)
  cv2.imwrite("output%d.jpg"%(count),result[0])
  
  result = (lr[0] + 1) * 127.5
  result = np.array(result, dtype=np.uint8)
  cv2.imwrite("input%d.jpg"%(count),result)
  
  model.save("model%d"%(count))


In [0]:
def conv_block(prevlayer, filters, kernel, prefix, strides=(1,1)):
	conv = Conv2D(filters, kernel, padding="same",  strides=strides,
				  name=prefix + "_conv")(prevlayer)

	conv = BatchNormalization(name=prefix + "_bn")(conv)
	conv = Activation('relu', name=prefix + "_activation")(conv)
	#conv = LeakyReLU(0.2, name=prefix + "_activation")(conv)
	return conv


def UNET():
  # Low resolution image input
  img_input = Input(shape=(240,480,3))

  conv1 = conv_block(img_input, 128, (3,3), "conv1_1")
  pool1 = MaxPooling2D(2, strides=(2,2), padding="same", name="pool1")(conv1)

  conv2 = conv_block(pool1, 256, (3,3), "conv2_1")
  pool2 = MaxPooling2D(2, strides=(2,2), padding="same", name="pool2")(conv2)

  conv3 = conv_block(pool2, 512, (3,3), "conv3_1")
  pool3 = MaxPooling2D(2, strides=(2,2), padding="same", name="pool3")(conv3)

  #####################################################
  conv4 = conv_block_simple_down(pool3, 512, (3,3), "conv4_1")
  #####################################################

  up5 = concatenate([UpSampling2D()(conv4), conv3])
  conv5 = conv_block(up5, 512, (3,3), "conv5_1")

  up6 = concatenate([UpSampling2D()(conv5), conv2])
  conv6 = conv_block(up6, 256, (3,3), "conv6_1")

  up7 = concatenate([UpSampling2D()(conv6), conv1])
  conv7 = conv_block(up7, 128, (3,3), "conv7_1")

  conv8 = conv_block(conv7, 64, (3,3), "conv7_2")

  prediction = Conv2D(3, (3,3), activation="tanh", padding="same", name="prediction", kernel_initializer="Orthogonal")(conv8)

  model = Model(img_input, prediction)
  return model

In [0]:
model = UNET()
optimizer = keras.optimizers.Adam(0.0001)
model.compile(loss='mse',
            optimizer=optimizer)
model.fit(x=lr,y=hr,batch_size=8,epochs=20,verbose=1)
pred(0)
model.fit(x=lr,y=hr,batch_size=8,epochs=20,verbose=1)
pred(1)
model.fit(x=lr,y=hr,batch_size=8,epochs=20,verbose=1)
pred(2)
model.fit(x=lr,y=hr,batch_size=8,epochs=20,verbose=1)
pred(3)
model.fit(x=lr,y=hr,batch_size=8,epochs=50,verbose=1)
pred(4)
model.fit(x=lr,y=hr,batch_size=8,epochs=50,verbose=1)
pred(5)
model.fit(x=lr,y=hr,batch_size=8,epochs=100,verbose=1)
pred(6)

Epoch 1/20
660/660 [==============================] - 169s 256ms/step - loss: 0.0512
Epoch 2/20
660/660 [==============================] - 133s 202ms/step - loss: 0.0358
Epoch 3/20
660/660 [==============================] - 133s 202ms/step - loss: 0.0331
Epoch 4/20
660/660 [==============================] - 133s 201ms/step - loss: 0.0332
Epoch 5/20
660/660 [==============================] - 133s 202ms/step - loss: 0.0318
Epoch 6/20
660/660 [==============================] - 134s 202ms/step - loss: 0.0289
Epoch 7/20
660/660 [==============================] - 133s 202ms/step - loss: 0.0311
Epoch 8/20
660/660 [==============================] - 133s 202ms/step - loss: 0.0264
Epoch 9/20
660/660 [==============================] - 133s 202ms/step - loss: 0.0273
Epoch 10/20
660/660 [==============================] - 134s 202ms/step - loss: 0.0273
Epoch 11/20
660/660 [==============================] - 134s 203ms/step - loss: 0.0260
Epoch 12/20
660/660 [==============================] - 133s 202

KeyboardInterrupt: ignored